<a href="https://colab.research.google.com/github/ShlokShar/CourseConnect/blob/main/ProblemSolverModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.4 MB/s eta 0:00:00


In [44]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForNextSentencePrediction, TrainingArguments, Trainer
from transformers import BertGenerationConfig, BertGenerationEncoder, BertGenerationDecoder, BertTokenizer, EncoderDecoderModel
from datasets import Dataset
from datasets import load_dataset
import torch
import pandas as pd
import numpy as np

In [45]:
categories = [
    'algebra',
    'counting_and_probability',
    'geometry',
    'intermediate_algebra',
    'number_theory',
    'prealgebra',
    'precalculus',
]

combined_train_df = pd.DataFrame()
combined_test_df = pd.DataFrame()

split_train = "train"
split_test = "test"

In [46]:
for subject in categories:
    dataset = load_dataset('baber/hendrycks_math', subject)

    data_split_train = dataset[split_train]
    data_split_test = dataset[split_test]

    train_df = pd.DataFrame(data_split_train)
    test_df = pd.DataFrame(data_split_test)

    combined_train_df = pd.concat([combined_train_df, train_df], ignore_index=True)
    combined_test_df = pd.concat([combined_test_df, test_df], ignore_index=True)

In [47]:
print(combined_train_df)

                                                problem    level         type  \
0     Let \[f(x) = \left\{\n\begin{array}{cl} ax+3, ...  Level 5      Algebra   
1     A rectangular band formation is a formation wi...  Level 5      Algebra   
2     What is the degree of the polynomial $(4 +5x^3...  Level 3      Algebra   
3     Evaluate $\left\lceil3\left(6-\frac12\right)\r...  Level 3      Algebra   
4     Sam is hired for a 20-day period. On days that...  Level 3      Algebra   
...                                                 ...      ...          ...   
7495  Find the number of real solutions of the equat...  Level 3  Precalculus   
7496  Let $A,$ $B,$ $C$ be the angles of a triangle....  Level 2  Precalculus   
7497  Let $G$ be the centroid of triangle $ABC,$ and...  Level 2  Precalculus   
7498  If angle $A$ lies in the second quadrant and $...  Level 2  Precalculus   
7499  The real numbers $a$ and $b$ satisfy\n\[\begin...  Level 2  Precalculus   

                           

In [48]:
encoder = BertGenerationEncoder.from_pretrained("bert-large-uncased", bos_token_id=101, eos_token_id=102)

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.


In [49]:
decoder = BertGenerationDecoder.from_pretrained(
    "bert-large-uncased", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102
)


You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of BertGenerationDecoder were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['bert.encoder.layer.14.crossattention.self.query.bias', 'bert.encoder.layer.11.crossattention.output.dense.weight', 'bert.encoder.layer.14.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.17.crossattention.self.value.weight', 'bert.encoder.layer.2.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.11.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.2.crossattention.self.query.bias', 'bert.encoder.layer.18.crossattention.self.query.bias', 'bert.encoder.layer.5.crossattention.self.key.weight', 'bert.encoder.layer.4.crossattention.self.query.bias', 'bert.encoder.layer.22.crossattention.self.key.bias', 'bert.encoder.layer.9.crossattention.self.key.weight', 'bert.encode

In [50]:
bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)


In [51]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

In [57]:
def tokenize_problem(examples):
    return tokenizer(examples['problem'], padding="max_length", truncation=True)

def tokenize_label(examples):
    return tokenizer(examples['type'], padding="max_length", truncation=True)

In [58]:
train_dataset = Dataset.from_pandas(combined_train_df)
test_dataset = Dataset.from_pandas(combined_test_df)

In [59]:
tokenized_train_dataset_inputs = train_dataset.map(tokenize_problem, batched=True)
tokenzied_test_dataset_inputs = test_dataset.map(tokenize_problem, batched=True)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [60]:
tokenized_train_dataset_labels = train_dataset.map(tokenize_label, batched=True)
tokenzied_test_dataset_labels = test_dataset.map(tokenize_label, batched=True)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [61]:
loss = bert2bert(input_ids=tokenized_train_dataset_inputs, decoder_input_ids=tokenized_train_dataset_labels, labels=tokenized_train_dataset_labels).loss
loss.backward()

TypeError: ignored

In [ ]:
trainData = pd.read_csv('sent_train.csv')
validData = pd.read_csv('sent_valid.csv')

train_dataset = Dataset.from_pandas(trainData)
valid_dataset = Dataset.from_pandas(validData)

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_valid_dataset = valid_data.set.map(tokenize_function, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased ", num_labels=3)

In [ ]:
training_args = TrainingArguments(
    'test-trainer',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
)